## tokenize function

In [5]:
import os
from glob import glob
import json
import re
import argparse

from transformers import BertTokenizer

# 문장 하나에 대해 processing하는 함수 클래스
class EntityPosMarker:
    """ 한국어 문장 하나에 대해 entity의 위치를 special token으로 나타낸 후,
        tokenize하여 BERT에 들어갈 수 있는 input ids 형태로 변환.
    
    Attributes:
        tokenizer: BertTokenizer(bert-base-multilingual-cased model을 사용).
        args: Args from command line.
    """
    def __init__(self, args=None):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
        self.err = 0
        self.args = args
    
    def tokenize(self, sentence, subj_pos_range, obj_pos_range):
        """ Entity Marker를 special token 형태로 추가한 후 BERT-input ids로 변환하는 함수.

        Args:
            sentence: 한국어 문장 (type: str)
            subject_pos_range: subject entity의 위치 인덱스 범위 (인덱스는 문자열 인덱스)
            object_pos_range: object entity의 위치 인덱스 범위 (인덱스는 문자열 인덱스)

        Returns:
            tokenized_input_ids: BERT-input ids
            subj_marker_start: subject entity marker 시작 위치 인덱스
            subj_marker_end: subject entity marker 끝 위치 인덱스
            obj_marker_start: object entity marker 시작 위치 인덱스
            obj_marker_end: object entity marker 끝 위치 인덱스

        Example:
            sentence: "한국은 동아시아의 한반도에 위치하고 있다."
            subj_pos_range: [0, 2] ("한국")
            obj_pos_range: [4, 8] ("동아시아")

            1. tokenizer.tokenize(sentence)
                tokens = ['한국', '##은', '동', '##아', '##시아', '##의', '한', '##반', '##도에', '위', '##치', '##하고', '있다', '.']
            2. special token 추가
                tokenized_sentence = ['[CLS]', '[unused1]', '한국', '[unused2]', '##은', '[unused3]', '동', '##아', '##시아', 
                                    '[unused4]', '##의', '한', '##반', '##도에', '위', '##치', '##하고', '있다', '.', '[SEP]']
            3. tokenized sentence를 BERT-input ids로 변환 및 entities 위치 찾아 return
                tokenized_input_ids = [101, 1, 48556, 2, 10892, 3, 9095, 16985, 46861, 4, 10459, 9954, 30134, 108521,
                                    9619, 18622, 12453, 11506, 119, 102]
        """
        # subj_name, obj_name
        subj_name = sentence[subj_pos_range[0]:subj_pos_range[1]]
        obj_name = sentence[obj_pos_range[0]:obj_pos_range[1]]

        subj_start_idx, subj_end_idx = [], []
        obj_start_idx, obj_end_idx = [], []

        # 1. tokenizer.tokenize(sentence)
        tokens = self.tokenizer.tokenize(sentence)

        # subj, obj token 위치 찾기
        for i, token in enumerate(tokens):
            if token[0] == subj_name[0]:
                subj_start_idx.append(i)
            if token[-1] == subj_name[-1]:
                subj_end_idx.append(i)
        
            if token[0] == obj_name[0]:
                obj_start_idx.append(i)
            if token[-1] == obj_name[-1]:
                obj_end_idx.append(i)
        
        # subj token idx
        subj_flag = False
        for i in subj_start_idx:
            for j in subj_end_idx:
                # tmp_tokens = ['한국']
                tmp_tokens = tokens[i:j+1]
                subj_cand = " ".join(tmp_tokens)
                subj_cand = re.sub(" ##", "", subj_cand)
                if subj_cand == subj_name:
                    subj_token_start, subj_token_end = i, j+1
                    subj_flag = True
                    break
            if subj_flag:
                break

        # obj token idx
        obj_flag = False
        for i in obj_start_idx:
            for j in obj_end_idx:
                # tmp_tokens = ['동', '##아', '##시아']
                tmp_tokens = tokens[i:j+1]
                obj_cand = " ".join(tmp_tokens)
                obj_cand = re.sub(" ##", "", obj_cand)
                if obj_cand == obj_name:
                    obj_token_start, obj_token_end = i, j+1
                    obj_flag = True
                    break
            if obj_flag:
                break

        # subj_tokens = tokens[subj_token_start:subj_token_end]
        # obj_tokens = tokens[obj_token_start:obj_token_end]

        # 2. special token 추가
        tokenized_sentence = []
        for i, token in enumerate(tokens):
            if i == subj_token_start:
                tokenized_sentence.append("[unused1]")
            elif i == obj_token_start:
                tokenized_sentence.append("[unused3]")

            if i == subj_token_end:
                tokenized_sentence.append("[unused2]")
            elif i == obj_token_end:
                tokenized_sentence.append("[unused4]")

            tokenized_sentence.append(token)

        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"]

        # 3. entity marker token 위치 찾기
        try:
            subj_marker_start = tokenized_sentence.index("[unused1]")
            subj_marker_end = tokenized_sentence.index("[unused2]")
        except:
            self.err += 1
            subj_marker_start = 0
            subj_marker_end = 2
        
        try:
            obj_marker_start = tokenized_sentence.index("[unused3]")
            obj_marker_end = tokenized_sentence.index("[unused4]")
        except:
            self.err += 1
            obj_marker_start = 0
            obj_marker_end = 2

        tokenized_input_ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        return tokenized_input_ids, subj_marker_start, subj_marker_end, obj_marker_start, obj_marker_end
        

## train data 확인

In [6]:
train_data = []
with open("../data/train.txt") as f:
    lines = f.readlines()
    for line in lines:
        item = json.loads(line)
        train_data.append(item)

In [7]:
print(f"Train data의 개수: {len(train_data)}")

Train data의 개수: 2685657


In [8]:
train_data[0]

{'file_name': '/data/modified_KBN_data/wikipedia_0001.json',
 'sentence': '제임스 얼 "지미"카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.',
 'subj': {'name': '제임스 얼', 'pos': [0, 5], 'type': 'PS'},
 'obj': {'name': '카터 주니어', 'pos': [10, 16], 'type': 'PS'},
 'description': '항목 주제의 후임자',
 'confidence': 0.8254553079605103}

In [9]:
train_data[1]

{'file_name': '/data/modified_KBN_data/wikipedia_0001.json',
 'sentence': '제임스 얼 "지미"카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.',
 'subj': {'name': '제임스 얼', 'pos': [0, 5], 'type': 'PS'},
 'obj': {'name': '미국', 'pos': [44, 46], 'type': 'LC'},
 'description': '항목 주제의 국가',
 'confidence': 0.5040757060050964}

## EntityPosMarker 사용 예시

In [10]:
marker = EntityPosMarker()

example = train_data[0]

sentence = example['sentence']
subj_pos_range = example['subj']['pos']
obj_pos_range = example['obj']['pos']

In [12]:
print(marker.tokenize(sentence=sentence, subj_pos_range=subj_pos_range, obj_pos_range=obj_pos_range))

([101, 1, 9672, 36240, 12605, 9551, 2, 107, 9706, 22458, 107, 3, 9786, 21876, 9689, 25503, 12965, 4, 113, 117, 11416, 10954, 16650, 18329, 198, 114, 9043, 9311, 16323, 21928, 9768, 25387, 23545, 11303, 48506, 70672, 113, 10722, 10954, 198, 96318, 114, 30919, 119, 102], 1, 6, 11, 17)


In [13]:
tokenized_input_ids, subj_marker_start, subj_marker_end, obj_marker_start, obj_marker_end = marker.tokenize(sentence, subj_pos_range, obj_pos_range)

In [19]:
print(f"Input ids: \n{tokenized_input_ids}")

Input ids: 
[101, 1, 9672, 36240, 12605, 9551, 2, 107, 9706, 22458, 107, 3, 9786, 21876, 9689, 25503, 12965, 4, 113, 117, 11416, 10954, 16650, 18329, 198, 114, 9043, 9311, 16323, 21928, 9768, 25387, 23545, 11303, 48506, 70672, 113, 10722, 10954, 198, 96318, 114, 30919, 119, 102]


In [17]:
print(f"Subj entity marker 위치: [{subj_marker_start}, {subj_marker_end}]")
print(f"Obj entity marker 위치: [{obj_marker_start}, {obj_marker_end}]")

Subj entity marker 위치: [1, 6]
Obj entity marker 위치: [11, 17]


In [25]:
print(f"Original Sentence: {sentence}")
print(f"Subj name: {sentence[subj_pos_range[0]:subj_pos_range[1]]}")
print(f"Obj name: {sentence[obj_pos_range[0]:obj_pos_range[1]]}")

Original Sentence: 제임스 얼 "지미"카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.
Subj name: 제임스 얼
Obj name: 카터 주니어


In [22]:
print(marker.tokenizer.convert_ids_to_tokens(tokenized_input_ids))

['[CLS]', '[unused1]', '제', '##임', '##스', '얼', '[unused2]', '"', '지', '##미', '"', '[unused3]', '카', '##터', '주', '##니', '##어', '[unused4]', '(', ',', '1924', '##년', '10월', '1일', '~', ')', '는', '민', '##주', '##당', '출', '##신', '미국', '39', '##번째', '대통령', '(', '1977', '##년', '~', '1981년', ')', '이다', '.', '[SEP]']


원래 문장이 tokenize된 후 entity marker의 역할을 하는 special token이 추가된 형태로 BERT-input ids로 변환되는 것을 확인할 수 있습니다.